In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def seed_everything(seed=1358):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

In [ ]:
PATH = Path('../input/aptos2019-blindness-detection/')
train_images = PATH/'train_images'
test_images = PATH/'test_images'

df_train = pd.read_csv(PATH/'train.csv')
df_test = pd.read_csv(PATH/'test.csv')

In [ ]:
df_train.head()

In [ ]:
df_train['id_code'] = df_train['id_code'].apply(lambda x : str(x) + '.png')

In [ ]:
df_test['id_code'] = df_test['id_code'].apply(lambda x : str(x) + '.png')

In [ ]:
tfms = get_transforms(do_flip=True,flip_vert=True,max_rotate=360.0,max_lighting=0.2,max_zoom=1.1)

In [ ]:
data = ImageDataBunch.from_df(df=df_train,path=PATH,folder='train_images',ds_tfms=get_transforms(),valid_pct=0.2,seed=42,size=256,num_workers=4).normalize(imagenet_stats)

In [ ]:
data

In [ ]:
kappa = KappaScore()
kappa.weights = "quadratic"

In [ ]:
if not os.path.exists('/tmp/.cache/torch/checkpoints/'):
        os.makedirs('/tmp/.cache/torch/checkpoints/')
!cp '../input/resnet50/resnet50.pth' '/tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth'

In [ ]:
learn = cnn_learner(data,models.resnet50,metrics=[kappa], model_dir=Path("/kaggle/working/"),path=Path("."))

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(4,max_lr=slice(1e-8,1e-6))

In [ ]:
learn.save('stage-2')

In [ ]:
learn.load('stage-2')